In [1]:
import torch
import urllib.request
import math
import seaborn as sns

import tensorflow as tf
tf.random.set_seed(7)
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from io import StringIO
from scipy.io import arff
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

import keras
from keras.layers import LSTM, Dropout, Dense
import tensorflow as tf
import numpy as np
import pandas as pd

from sklearn import svm

2023-06-20 03:54:51.842388: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-20 03:54:52.493038: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
ftpstream = urllib.request.urlopen('http://www.ece.uah.edu/~thm0009/icsdatasets/IanArffDataset.arff')
arffdata, arffmeta = arff.loadarff(StringIO(ftpstream.read().decode('utf-8')))

data_df = pd.DataFrame(arffdata)
data_df = data_df[data_df["time"] < 1418780000]

data_df['command response'] = data_df['command response'].str.decode("utf-8")
data_df['binary result'] = data_df['binary result'].str.decode("utf-8")
data_df['categorized result'] = data_df['categorized result'].str.decode("utf-8")
data_df['specific result'] = data_df['specific result'].str.decode("utf-8")

data_df['command response'] = data_df['command response'].astype(float)
data_df['binary result']  = data_df['binary result'] .astype(float)
data_df['categorized result'] = data_df['categorized result'].astype(float)
data_df['specific result'] = data_df['specific result'].astype(float)

data_df['nc'] = data_df.isnull().sum(axis=1)
#data_df.isnull().sum(axis=1)
data_df = data_df.sort_values('time')

nancols = data_df.columns[data_df.isna().any()].tolist()
for col in nancols:
  data_df[col] = data_df[col].interpolate()

#inf nan etc
not_numbers = data_df.isin([np.inf, -np.inf]).sum().sum() + data_df.isna().sum().sum()

data_df = data_df.dropna() # dla małej części nie zadziałało ale życie
not_numbers = data_df.isin([np.inf, -np.inf]).sum().sum() + data_df.isna().sum().sum()

print(data_df.shape)

(97598, 21)


In [3]:
datacols = ['address', 'function', 'length', 'setpoint', 'gain', 'reset rate',
            'deadband', 'cycle time', 'rate', 'system mode', 'control scheme', 'pump',
            'solenoid', 'pressure measurement', 'crc rate','nc']
additional_cols = ['specific result', 'categorized result', 'binary result', 'time', 'command response', 'nc']

#skalowanie
scaler = MinMaxScaler()
scaler.fit(data_df[datacols])
data_df[datacols] = pd.DataFrame(scaler.transform(data_df[datacols]))

attack_df = data_df[data_df['binary result'] == 1]
normal_df = data_df[data_df['binary result'] == 0]

In [13]:
rec_count = len(data_df)
seq_length = 30

def prepare_data(X, y, seq_length = 20, step = 1):
    X_local = []
    y_local = []
    for start in range(0, len(data_df) - seq_length, step):
        end = start + seq_length
        X_local.append(X[start:end])
        y_local.append(y[end-1])
    return np.array(X_local), np.array(y_local)

a1 = data_df[datacols]
a2 = data_df['binary result']
a2[a2>0]=1

X_sequence, y = prepare_data(a1.values, a2)

X_sequence.shape, y.shape

((97578, 20, 16), (97578,))

In [14]:
model = keras.Sequential()

model.add(Conv1D(filters=8, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss="binary_crossentropy"
              , metrics=[keras.metrics.binary_accuracy]
              , optimizer="adam")
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [15]:
train_size = int(len(X_sequence) * 0.7)

In [16]:
X_train, y_train = X_sequence[:train_size], y[:train_size]
X_test, y_test = X_sequence[train_size:], y[train_size:]

In [17]:
model.fit(X_train, y_train, batch_size=64, epochs=20)

Epoch 1/20


2023-06-20 03:59:03.769298: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-20 03:59:03.770996: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-20 03:59:03.771872: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1068/1068 [==============================] - 8s 6ms/step - loss: 0.4315 - binary_accuracy: 0.8094
Epoch 2/20
1068/1068 [==============================] - 7s 6ms/step - loss: 0.3571 - binary_accuracy: 0.8503
Epoch 3/20
1068/1068 [==============================] - 7s 6ms/step - loss: 0.3198 - binary_accuracy: 0.8666
Epoch 4/20
1068/1068 [==============================] - 7s 6ms/step - loss: 0.3043 - binary_accuracy: 0.8730
Epoch 5/20
1068/1068 [==============================] - 7s 6ms/step - loss: 0.2937 - binary_accuracy: 0.8783
Epoch 6/20
1068/1068 [==============================] - 7s 6ms/step - loss: 0.2852 - binary_accuracy: 0.8819
Epoch 7/20
1068/1068 [==============================] - 7s 6ms/step - loss: 0.2783 - binary_accuracy: 0.8844
Epoch 8/20
1068/1068 [==============================] - 7s 6ms/step - loss: 0.2720 - binary_accuracy: 0.8874
Epoch 9/20
1068/1068 [==============================] - 7s 6ms/step - loss: 0.2633 - binary_accuracy: 0.8904
Epoch 10/20
1068/1068 [=======

In [18]:
model.evaluate(X_test, y_test)

 22/915 [..............................] - ETA: 2s - loss: 1.2456 - binary_accuracy: 0.7472  

2023-06-20 04:02:59.620396: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-20 04:02:59.621729: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-20 04:02:59.622538: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

915/915 [==============================] - 2s 2ms/step - loss: nan - binary_accuracy: 0.8284


[nan, 0.828448474407196]

In [19]:
y_test_prob = model.predict(X_test, verbose=1)
y_test_pred = np.where(y_test_prob > 0.4, 1, 0)
accuracy = accuracy_score(y_test,y_test_pred)
precision = precision_score(y_test,y_test_pred)
recall = recall_score(y_test,y_test_pred)
f1 = f1_score(y_test,y_test_pred)

print(accuracy)
print(precision)
print(recall)
print(f1)

confusion_matrix(y_test, y_test_pred)

 52/915 [>.............................] - ETA: 1s

2023-06-20 04:03:04.875882: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-20 04:03:04.877069: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-20 04:03:04.877752: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

915/915 [==============================] - 2s 2ms/step
0.8124615699938512
0.5936957061305534
0.6834285714285714
0.6354097489706468


array([[19000,  3274],
       [ 2216,  4784]])